In [62]:
import requests
import datetime

def populate_list(number_days_window):
    populated_list = []
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live')
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live', verify=False)
    all_days = requests.get('https://api.covid19api.com/total/country/uk')
    first=True
    for day in all_days.json():
        date_time_obj = datetime.datetime.strptime(day['Date'][:10], '%Y-%m-%d')
        if first:
            ordinal_first = date_time_obj.toordinal()
        n = date_time_obj.toordinal() - ordinal_first
        delta = 0
        fatality_rate = 0
        if (day['Confirmed'] < 1000):
            continue
        if n>number_days_window:
            # delta mean
            delta = (day['Confirmed'] - populated_list[n-number_days_window][2]) / number_days_window
        first=False
        if day['Confirmed'] > 0:
            fatality_rate = day['Deaths']/day['Confirmed']
        populated_list.append((n, day['Active'], day['Confirmed'], delta, fatality_rate, date_time_obj))
    return populated_list

In [63]:
import pandas as pd
import numpy as np

# Si fatality rate sube, se esta subtesteando, hay mas casos de lo que se cree.
# jump days cada cuantos dias sacas una foto (mirando pa atras)

def process_means(number_days_window, jump_days, total_jumps):
    my_populated_list = populate_list(number_days_window)
    means_list = []
    fatality_rate = []
    active_cases = []
    total_lapse_day = jump_days * total_jumps
    df = pd.DataFrame (my_populated_list,columns=['day-number', 'active', 'cases-count','delta', 'fatality_rate','date'])
    
    for n in range(total_lapse_day,0,-jump_days):
        try:
            mean = round(df[-(n):-(n-(jump_days))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days))]['active'].mean())
        except:
            mean = round(df[-(n):-(n-(jump_days-1))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days-1))]['active'].mean())
        means_list.append(mean)
        fatality_rate.append(df[-n:-(n-1)]['fatality_rate'].mean())
        active_cases.append(active)
    return means_list, fatality_rate, active_cases



In [64]:
import statistics
total_jumps = 3
a = range(total_jumps)

#print(process_means(30,14,total_jumps))

#means_list, fatality_rate, active_cases = process_means(30,14,total_jumps)
from scipy.stats import linregress
for params in [process_means(30,14,total_jumps), process_means(14,14,total_jumps), process_means(10,14,total_jumps), process_means(7,14,total_jumps), process_means(5,14,total_jumps)]:
    means_list, fatality_rate, active_cases = params
    for result in [means_list, fatality_rate, active_cases]:
        print(result)
        print(linregress(a, result))
        print(statistics.pstdev(result), statistics.stdev(result),statistics.pvariance(result),statistics.variance(result))
    print ("...----...")

[19185, 19754, 32501]
LinregressResult(slope=6657.999999999999, intercept=17155.333333333332, rvalue=0.8843002320107634, pvalue=0.309271886084176, stderr=3515.4857890956328)
6147.498371062999 7529.117101847555 37791736.222222224 56687604.333333336
[0.03583510891926649, 0.03440244027976384, 0.030477753127503542]
LinregressResult(slope=-0.0026786778958814723, intercept=0.03625044533805943, rvalue=-0.9657783699899839, pvalue=0.16702898162989097, stderr=0.0007193837795830816)
0.002264630697223609 0.0027735948320205734 5.128552194807488e-06 7.692828292211232e-06
[1671982, 2000136, 2589463]
LinregressResult(slope=458740.5, intercept=1628453.1666666667, rvalue=0.9867620671199917, pvalue=0.10370160354291161, stderr=75394.15092753104)
379584.97014004947 464894.745436355 144084749556.22223 216127124334.3333
...----...
[15794, 23936, 43982]
LinregressResult(slope=14094.0, intercept=13810.0, rvalue=0.9715389368517406, pvalue=0.15224959205486618, stderr=3436.3888022166448)
11844.818951761146 14506.

# Algunas reglas

Si fatality_rate > fatality_rate_mean:
    Yellow Warning Casos pueden ser mayor de lo reflejado pues pueden que estén subtesteando.

Si fatality_rate > fatality_rate_mean y active_cases y means_list en crecimiento:

    Rojo Warning 

means_list, fatality_rate, active_cases


In [65]:
from random import choice
from experta import *


class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])


engine1 = RobotCrossStreet()
engine1.reset()
engine1.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine1.run()


Don't walk
